In [3]:
import data_cleaning as cl
import dirty_data as dd
import pandas as pd
import numpy as np
import datetime
import CityDict

IndentationError: expected an indented block (dirty_data.py, line 626)

In [2]:
df = pd.DataFrame([])

NameError: name 'pd' is not defined

### Dictionaries for Data Cleaning and Categorizations

In [5]:
chi = CityDict.cities

### Prepare Sheriff Custody Data for May 31st, 2016

In [6]:
data1 = dd.DirtyData("/Users/annahazard/Dropbox/Injustice_Watch/Cook County/Original Data/Second Request/Cook_County_May_31_2016.csv", 5, 31, 2016)
data1.uppercase("ChargeDescription")
data1.uppercase("Inmatefirst")
data1.uppercase("InmateLast")
data1.uppercase("City")
data1.fill_zeroes("CurrentBond")
data1.fill_zeroes("TotalBond")
data1.numbify_currency("CurrentBond")
data1.numbify("TotalBond")
data1.process_dates("Last Admission Date")
data1.simuldict["City"] = chi
data1.build_replacements_from_simdict("City")
data1.check_label_validity("City")

{}

In [7]:
data1.read_crimes("StandardizedCharges.csv")

In [8]:
data1.standardize_crime("Rows_with_undefined_crimes")

All rows successfully processed


In [9]:
data1.replace_column("City")
data1.init_individuals()

In [10]:
data1.post_process_individuals()

### Write out db of individuals

In [11]:
f = open("CookCounty05312016.py", "w")
f.write(str(data1.individual_dict))

6059468

### Count Inmates by Reason for Custody

In [12]:
count_1 = 0
indices_1 = []
count_2 = 0
indices_2 = []
count_3 = 0
indices_3 = []
count_4 = 0
indices_4 = []
count_5 = 0
indices_5 = []
count_6 = 0
indices_6 = []
count_7 = 0
indices_7 = []
count_8 = 0
indices_8 = []

for iid, info in data1.individual_dict.items():
    if info["Reason for Custody"] == "SERVING SENTENCE":
        count_1 += 1
        indices_1.extend(info["Indices"])
    elif info["Reason for Custody"] == "PRETRIAL, FAILURE TO PLACE":
        count_2 += 1
        indices_2.extend(info["Indices"])
    elif info["Reason for Custody"] == "PRETRIAL, NO BOND":
        count_3 += 1
        indices_3.extend(info["Indices"])
    elif info["Reason for Custody"] == "PRETRIAL, FAILURE TO PAY":
        count_4 += 1
        indices_4.extend(info["Indices"])
    elif info["Reason for Custody"] == "WARRANT":
        count_5 += 1
        indices_5.extend(info["Indices"])
    elif info["Reason for Custody"] == "UNFIT AWAITING PLACEMENT":
        count_6 += 1
        indices_6.extend(info["Indices"])
    elif info["Reason for Custody"] == "NO REASON TO HOLD":
        count_7 += 1
        indices_7.extend(info["Indices"])
    elif info["Reason for Custody"] == "STATUS OF ALL CHARGES UNKNOWN":
        count_8 += 1
        indices_8.extend(info["Indices"])

In [13]:
NoPaySubset = data1.make_subset(indices_4)
NoPaySubset.count_by_highest_charge()

In [14]:
NoBondSubset = data1.make_subset(indices_3)
NoBondSubset.count_by_highest_charge()

In [15]:
NoPaySubset.FX

2648

In [16]:
data1.count_by_highest_charge()

In [17]:
data1.FX

3693

In [19]:
len(NoBondSubset.individual_dict)

1081

In [30]:
indices = []
count = 0
for iid, info in NoBondSubset.individual_dict.items():
    if info["Charge Rank"] < 12:
        count += 1
        indices.extend(info["Indices"])

In [31]:
count

373

In [27]:
row_list = []
for i in indices:
    row_list.append(data1.df.iloc[i])
    
NonXNoBond = pd.DataFrame(row_list)

In [29]:
NonXNoBond.to_csv("Lower Level Crimes No Bond.csv", index=False)

### Examine No-Bond Cases

In [ ]:
indices = []
for iid, info in data1.individual_dict.items():
    if info["Total Bond"] == 0:
        indices.extend(info["Indices"])
        
data = data1.df
row_list = []
for i in indices:
    row_list.append(data.iloc[i])
    
df = pd.DataFrame(row_list)
df.to_csv("../Output/No_Bond_Given.csv", index=False)

### Retail Theft Cases

In [ ]:
all_time = 0
count = 0
for iid, info in data1.individual_dict.items():
    if info["Highest Level Charge"] == 'RETAIL THEFT':
        count += 1
        all_time += info["Days in Custody from Last Admission"]
avg_time = all_time/count

In [ ]:
avg_time, count

In [ ]:
all_time = 0
count = 0
all_charges = 0
indices = []
data = data1.df

for iid, info in data1.individual_dict.items():
    if info["Highest Level Charge"] == 'RETAIL THEFT' \
    and info["Days in Custody from Last Admission"] > 120:
        count += 1
        all_charges += info["Number of Charges"]
        all_time += info["Days in Custody from Last Admission"]
        indices.extend(info["Indices"])

row_list = []
for i in indices:
    row_list.append(data.iloc[i])
    
df = pd.DataFrame(row_list)
df.to_csv("../Output/Retail_Theft_Over_120_Days.csv", index=False)
        
avg_numcharge = all_charges/count

In [ ]:
avg_numcharge, count

### DUI Cases

#### Regular DUI

In [ ]:
all_time = 0
count = 0
for iid, info in data1.individual_dict.items():
    if info["Highest Level Charge"] == 'DUI':
        count += 1
        all_time += info["Days in Custody from Last Admission"]
avg_time_dui = all_time/count

In [ ]:
avg_time, count

In [ ]:
all_time = 0
count = 0
all_charges = 0
indices = []
data = data1.df

for iid, info in data1.individual_dict.items():
    if info["Highest Level Charge"] == 'DUI' \
    and info["Days in Custody from Last Admission"] > 120:
        count += 1
        all_charges += info["Number of Charges"]
        all_time += info["Days in Custody from Last Admission"]
        indices.extend(info["Indices"])

row_list = []
for i in indices:
    row_list.append(data.iloc[i])
    
df = pd.DataFrame(row_list)
df.to_csv("../Output/DUI_Over_120_Days.csv", index=False)
        
avg_numcharge = all_charges/count

In [ ]:
count

#### Aggravated DUI

In [ ]:
all_time = 0
count = 0
for iid, info in data1.individual_dict.items():
    if info["Highest Level Charge"] == 'DUI: AGGRAVATED':
        count += 1
        all_time += info["Days in Custody from Last Admission"]
avg_time = all_time/count

In [ ]:
avg_time, count

In [ ]:
all_time = 0
count = 0
all_charges = 0
indices = []
data = data1.df

for iid, info in data1.individual_dict.items():
    if info["Highest Level Charge"] == 'DUI: AGGRAVATED' \
    and info["Days in Custody from Last Admission"] > 120:
        count += 1
        all_charges += info["Number of Charges"]
        all_time += info["Days in Custody from Last Admission"]
        indices.extend(info["Indices"])

row_list = []
for i in indices:
    row_list.append(data.iloc[i])
    
df = pd.DataFrame(row_list)
df.to_csv("../Output/Agg_DUI_Over_120_Days.csv", index=False)
        
avg_numcharge = all_charges/count

In [ ]:
count

### Get List of Unique Case Statuses

In [ ]:
len(data1.individual_dict)

In [ ]:
data1.get_unique_multi(["casedisposition"], write_to="Unique Case Dispositions.csv")

### Time vs. Charge

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="4", days_above=365, write_to="../Output/2016-07-06 Time vs Charge/At Least Class 3 Felony and Stay >1 year.csv")

In [ ]:
data1.individual_dict

In [ ]:
count

In [ ]:
count2, data2 = data1.find_stay_vs_class(charge_above="NONE", charge_below="A", days_above=365, write_to="../Output/2016-07-21 Time vs Charge/At or Below Class A Misdemeanor and Stay >1 year.csv")

In [ ]:
count2

In [ ]:
count3, data3 = data1.find_stay_vs_class(charge_below="B", charge_above="NONE", days_above=186, write_to="../Output/2016-07-06 Time vs Charge/At or Below Class B Misdemeanor and Stay >6 Months.csv")

In [ ]:
count3

In [ ]:
count4, data4 = data1.find_stay_vs_class(charge_below="C", charge_above="NONE", days_above=30, write_to="../Output/2016-07-06 Time vs Charge/At or Below Class C Misdemeanor and Stay >30 Days.csv")

In [ ]:
count4

In [ ]:
count5, data5 = data1.find_stay_vs_class(charge_above="NONE", charge_below="C", days_above=120, write_to="../Output/2016-07-06 Time vs Charge/At or Below Class C Misdemeanor and Stay >120 Days.csv")

In [ ]:
count5

In [ ]:
count6, data6 = data1.find_stay_vs_class(charge_above="A", days_above=365, write_to="../Output/2016-07-06 Time vs Charge/At Least Class 4 Felony and Stay >1 year.csv")

In [ ]:
count6

In [ ]:
count7, data7 = data1.find_stay_vs_class(charge_above="4", days_above=730, write_to="../Output/2016-07-06 Time vs Charge/At Least Class 3 Felony and Stay >2 years.csv")

In [ ]:
count7

In [ ]:
count8, data8 = data1.find_stay_vs_class(charge_above="4", days_above=1095, write_to="../Output/2016-07-06 Time vs Charge/At Least Class 3 Felony and Stay >3 years.csv")

In [ ]:
count8

In [ ]:
coun9, data9 = data1.find_stay_vs_class(charge_above="4", days_above=1460, write_to="../Output/2016-07-06 Time vs Charge/At Least Class 3 Felony and Stay >4 years.csv")

In [ ]:
coun9

In [ ]:
dead_count = 0
dead_total = 0
indices = []
for temp_id, info in data1.individual_dict.items():
    if info["Dead Time"]>0 and info["Highest Level Charge Class"] != "NONE" and info["Highest Level Charge Class"] != "CONTEMPT":
        dead_count += 1
        dead_total += info["Dead Time"]
        indices.extend(info["Indices"])

row_list = []
for index in indices:
    row_list.append(data1.df.iloc[index])
    mini_df = pd.DataFrame(row_list)
    mini_df.to_csv("../Output/2016-07-06 Time vs Charge/People serving dead jail time.csv", index=False)

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="1", charge_below= "X", days_above=365*4, write_to="../Output/2016-07-06 Time vs Charge/Highest Charge Class X, Stay >4 Years.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="2", charge_below= "1", days_above=365*4, write_to="../Output/2016-07-06 Time vs Charge/Highest Charge Class 1, Stay >4 Years.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="3", charge_below= "2", days_above=365*4, write_to="../Output/2016-07-21 Time vs Charge/Highest Charge Class 2, Stay >4 Years.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="4", charge_below= "3", days_above=365*2, write_to="../Output/2016-07-21 Time vs Charge/Highest Charge Class 3, Stay >2 Years.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="A", charge_below= "4", days_above=365, write_to="../Output/2016-07-21 Time vs Charge/Highest Charge Class 4, Stay >1 Year.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="B", charge_below= "A", days_above=14, write_to="../Output/2016-07-21 Time vs Charge/Highest Charge Class A, Stay >14 Days.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="C", charge_below= "B", days_above=14, write_to="../Output/2016-07-21 Time vs Charge/Highest Charge Class B, Stay >14 Days.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_class(charge_above="LOCAL", charge_below= "C", days_above=14, write_to="../Output/2016-07-21 Time vs Charge/Highest Charge Class C or Below, Stay >14 Days.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_bond(bond_above=1500, bond_below=2500, days_above=4, days_below=float("inf"),  write_to="../Output/2016-07-21 Time vs Charge/Bond between 1500 and 2500, Stay >4 Days.csv")

In [ ]:
count

In [ ]:
count, data = data1.find_stay_vs_bond(bond_above=0, bond_below=1500, days_above=4, days_below=float("inf"),  write_to="../Output/2016-07-21 Time vs Charge/Bond at or below 1500, Stay >4 Days.csv")

In [ ]:
count

### Prepare Sheriff EM Data for May 31st, 2016

In [ ]:
data2 = dd.DirtyData("Data/Original_May_31_Sheriff_Data_EM.csv", 5, 31, 2016)
data2.uppercase("ChargeDescription")
data2.uppercase("Inmatefirst")
data2.uppercase("InmateLast")
data2.uppercase("City")
data2.fill_zeroes("CurrentBond")
data2.fill_zeroes("TotalBond")
data2.numbify("CurrentBond")
data2.numbify("TotalBond")
data2.process_dates("Last Admission Date")
data2.simuldict["City"] = chi
data2.build_replacements_from_simdict("City")
data2.check_label_validity("City")

In [ ]:
data2.replace_column("City")
data2.init_individuals()

### Defining Charges Categories

In [ ]:
data1.get_unique_multi(["Statute"], write_to="Unique Statutes CCDOC.csv")

### Bond vs. Stay Duration

In [ ]:
discrep1 = data1.detect_bond_discrepencies(remove=True)

In [ ]:
bond_buckets = [1000, 10000, 20000, 30000, 40000, 50000, 75000, 90000, 100000, 150000, 200000, 250000, 300000, 500000]

In [ ]:
time_buckets = np.arange(0, 3650, 120)

In [ ]:
column_headers = []
row_headers = []
numb_grid = []
dict_grid = []

for i, low_b in enumerate(bond_buckets):
    if len(bond_buckets) <= i+1:
        high_b = float("inf")
    else:
        high_b = bond_buckets[i+1]
        
    row_headers.append(str("%s to %s dollars" % (low_b, high_b)))
    numb_row = []
    dict_row = []
    
    for j, low_t in enumerate(time_buckets):
        if len(time_buckets) <= j+1:
            high_t = float("inf")
        else:
            high_t = time_buckets[j+1]
        if i==0:
            column_headers.append(str("%s to %s days in custody" % (low_t, high_t)))
        count, people_dict = data1.find_stay_vs_bond(bond_above = low_b, bond_below= high_b, \
                                                     days_above = low_t, days_below = high_t)
        numb_row.append(count)
        dict_grid.append(people_dict)
    numb_grid.append(numb_row)
    dict_grid.append(dict_row)

In [ ]:
counts = pd.DataFrame(numb_grid, index=row_headers, columns = column_headers)

In [ ]:
counts.to_csv("Jail Occupants by Time and Bail - Smaller.csv")

In [ ]:
count, people_dict = data1.find_stay_vs_bond(bond_above = -1, bond_below= float("inf"), \
                                                     days_above = 365, days_below = float("inf"))

In [ ]:
count

In [ ]:
new_grid = []
for temp_id, info in people_dict.items():
    row = []
    row.append(info["Last Name"])
    row.append(info["First Name"])
    row.append(info["Total Bond"])
    